30 Epoch GAN Test

In [18]:
import os
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tensorflow.keras import layers, losses
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import os
import logging
SEED = 0
keras.utils.set_random_seed(123)

In [19]:
csv_entry_data = pd.read_csv("onlinefraud.csv",sep=",")
credit_card_data=csv_entry_data
target_1_rows = credit_card_data[credit_card_data['isFraud'] == 1].head(20000)
target_0_rows = credit_card_data[credit_card_data['isFraud'] == 0].head(20000)
credit_card_data=pd.concat([target_1_rows, target_0_rows])
credit_card_data

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

Preprocessing

In [ ]:
#Preprocess
def standardize_and_normalization(func_df,colname):
    denom=(func_df[colname].std())
    numer=(func_df[colname]-func_df[colname].mean())
    func_df[colname] =numer/denom
Dummy_Results = pd.get_dummies(credit_card_data["type"],prefix='Type_')
credit_card_data.drop("type", inplace=True, axis=1)
credit_card_data=pd.concat([credit_card_data,Dummy_Results],axis='columns')
credit_card_data = credit_card_data.drop("isFlaggedFraud", axis=1)
credit_card_data = credit_card_data.drop("nameOrig", axis=1)
credit_card_data = credit_card_data.drop("nameDest", axis=1)
#one hot encode the actions
credit_card_data['Type__CASH_IN'] = credit_card_data['Type__CASH_IN'].replace({True: 1, False: 0})
credit_card_data['Type__CASH_OUT'] = credit_card_data['Type__CASH_OUT'].replace({True: 1, False: 0})
credit_card_data['Type__DEBIT'] = credit_card_data['Type__DEBIT'].replace({True: 1, False: 0})
credit_card_data['Type__PAYMENT'] = credit_card_data['Type__PAYMENT'].replace({True: 1, False: 0})
credit_card_data['Type__TRANSFER'] = credit_card_data['Type__TRANSFER'].replace({True: 1, False: 0})
#standardize/normalize the numerical data
standardize_and_normalization(credit_card_data,'amount')
standardize_and_normalization(credit_card_data,'oldbalanceOrg')
standardize_and_normalization(credit_card_data,'newbalanceOrig')
standardize_and_normalization(credit_card_data,'oldbalanceDest')
standardize_and_normalization(credit_card_data,'newbalanceDest')
credit_card_data=credit_card_data.fillna(value=0)
credit_card_data

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,Type__CASH_IN,Type__CASH_OUT,Type__DEBIT,Type__PAYMENT,Type__TRANSFER
2,1,-0.354846,-0.391393,-0.294012,-0.267946,-0.356414,1,0,0,0,0,1
3,1,-0.354846,-0.391393,-0.294012,-0.260288,-0.356414,1,0,1,0,0,0
251,1,-0.353038,-0.390421,-0.294012,-0.267946,-0.356414,1,0,0,0,0,1
252,1,-0.353038,-0.390421,-0.294012,-0.258473,-0.356414,1,0,1,0,0,0
680,1,-0.341108,-0.384007,-0.294012,-0.267946,-0.356414,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
20076,8,-0.257298,0.569947,0.950778,0.000553,0.172395,0,1,0,0,0,0
20077,8,0.080382,0.622453,1.238102,0.359858,-0.356414,0,1,0,0,0,0
20078,8,-0.251241,0.856486,1.306561,0.120871,-0.147650,0,1,0,0,0,0
20079,8,-0.318030,0.912248,1.330942,-0.140498,-0.356414,0,1,0,0,0,0


In [ ]:
def title_strip(data):
    dataframe_post_strip = pd.DataFrame(data)
    for c in range(dataframe_post_strip.shape[1]):
        mapping = {dataframe_post_strip.columns[c]: c}
        dataframe_post_strip = dataframe_post_strip.rename(columns=mapping)
    return dataframe_post_strip
entry_data_for_gan=credit_card_data.drop(columns=["isFraud"])
prepped_data = (entry_data_for_gan).values
X=prepped_data
y = (credit_card_data["isFraud"]).values
y_holder1=y
target_label1=y
X = KNNImputer().fit_transform(X)
credit_card_data = title_strip(StandardScaler().fit_transform(X))
credit_card_data
data_og=credit_card_data
data_og

,0,1,2,3,4,5,6,7,8,9,10
0,-0.547045,-0.354853,-0.391400,-0.294017,-0.267950,-0.356420,-0.380677,-0.614044,-0.126158,-0.7666,1.917608
1,-0.547045,-0.354853,-0.391400,-0.294017,-0.260292,-0.356420,-0.380677,1.628548,-0.126158,-0.7666,-0.521483
2,-0.547045,-0.353045,-0.390428,-0.294017,-0.267950,-0.356420,-0.380677,-0.614044,-0.126158,-0.7666,1.917608
3,-0.547045,-0.353045,-0.390428,-0.294017,-0.258477,-0.356420,-0.380677,1.628548,-0.126158,-0.7666,-0.521483
4,-0.547045,-0.341114,-0.384014,-0.294017,-0.267950,-0.356420,-0.380677,-0.614044,-0.126158,-0.7666,1.917608
...,...,...,...,...,...,...,...,...,...,...,...
28208,-0.512370,-0.257302,0.569957,0.950794,0.000553,0.172398,2.626897,-0.614044,-0.126158,-0.7666,-0.521483
28209,-0.512370,0.080383,0.622464,1.238124,0.359865,-0.356420,2.626897,-0.614044,-0.126158,-0.7666,-0.521483
28210,-0.512370,-0.251245,0.856501,1.306585,0.120873,-0.147652,2.626897,-0.614044,-0.126158,-0.7666,-0.521483
28211,-0.512370,-0.318036,0.912264,1.330965,-0.140500,-0.356420,2.626897,-0.614044,-0.126158,-0.7666,-0.521483


GAN 30 Epochs for synethtic data

In [ ]:
class Gan_Class():
    def __init__(self, data):
        self.data = data
        #change this number to change the number of epochs
        self.epoch_number = 5
    def train(self, generator, discriminator, gan):
        for epoch in range(self.epoch_number):
            generated_data = generator.predict(self.noise_input())
            labels = np.concatenate([np.ones(self.data.shape[0]), np.zeros(self.data.shape[0])])
            X = np.concatenate([self.data, generated_data])
            discriminator.trainable = True
            d_loss , _ = discriminator.train_on_batch(X, labels)
            noise = self.noise_input()
            g_loss = gan.train_on_batch(noise, np.ones(self.data.shape[0]))
            print('>%d, d1=%.3f, d2=%.3f' %(epoch+1, d_loss, g_loss))
        return generator    
    def GAN_Generator(self):
        model = tf.keras.Sequential(name="generator")
        model.add(tf.keras.layers.Dense(50, activation='relu',kernel_initializer='he_uniform',input_dim=self.data.shape[1]))
        model.add(tf.keras.layers.Dense(60, activation='gelu'))
        model.add(tf.keras.layers.Dense(self.data.shape[1], activation='linear'))
        return model
    def noise_input(self):
        noise = np.random.normal(0, 1, self.data.shape)
        return noise
    def GAN_Discriminator(self):
        model = tf.keras.Sequential(name="discriminator")
        model.add(tf.keras.layers.Dense(50, activation='gelu', kernel_initializer='he_uniform',input_dim=self.data.shape[1]))
        model.add(tf.keras.layers.Dense(60, activation='gelu'))
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        model.compile(loss='kld', optimizer='adam',metrics=['accuracy'])
        return model
    def GAN_Itself(self, generator, discriminator):
        generator.trainable = True
        discriminator.trainable = False
        model = tf.keras.Sequential(name="GAN")
        model.add(generator)
        model.add(discriminator)
        model.compile(loss='kld', optimizer='adam')
        return model
gan_train_data=credit_card_data[credit_card_data.index % 2 == 0]
model = Gan_Class(data=gan_train_data)
gen_V2 = model.GAN_Generator()
discy_V2 = model.GAN_Discriminator()
gan_model = model.GAN_Itself(generator=gen_V2, discriminator=discy_V2)
print(gan_model)
trained_model = model.train(generator=gen_V2, discriminator=discy_V2, gan=gan_model)
noise = np.random.normal(0, 1, gan_train_data.shape)
new_data = title_strip(data=trained_model.predict(noise))

<Sequential name=GAN, built=False>
107/441 ━━━━━━━━━━━━━━━━━━━━ 0s 479us/step

c:\Programming\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step
>1, d1=0.522, d2=0.725
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 444us/step
>2, d1=0.502, d2=0.697
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step
>3, d1=0.482, d2=0.670
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step
>4, d1=0.463, d2=0.644
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step
>5, d1=0.446, d2=0.619
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step
>6, d1=0.429, d2=0.595
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 439us/step
>7, d1=0.413, d2=0.572
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step
>8, d1=0.399, d2=0.550
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step
>9, d1=0.385, d2=0.529
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 429us/step
>10, d1=0.372, d2=0.508
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 432us/step
>11, d1=0.360, d2=0.488
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step
>12, d1=0.348, d2=0.469
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step
>13, d1=0.337, d2=0.451
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step
>14, d1=0.327, d2=0.433
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step
>15, d1=0.318, d2=0.416
441/

In [ ]:
new_data

,0,1,2,3,4,5,6,7,8,9,10
0,-1.434857,-0.112143,-1.262450,-0.181652,-0.379055,1.269093,-1.250588,0.333421,-0.740772,-0.129465,0.652690
1,0.074226,-0.251956,-0.317629,-0.444534,-0.947159,0.833016,-0.010900,0.394039,-0.233448,-0.452503,-0.419339
2,-2.113026,0.363721,-2.377197,0.261588,0.778938,1.619202,-1.927436,1.060402,-0.482880,1.141948,1.251349
3,-0.829484,0.482057,-1.242028,0.668319,-0.033884,2.348161,-0.790993,0.758014,-0.018492,-0.077402,0.398059
4,-0.372144,-0.091898,-1.026867,0.182926,-0.415799,1.445212,-0.618116,0.275018,-0.784466,-0.015670,0.600539
...,...,...,...,...,...,...,...,...,...,...,...
14102,-0.434718,0.162063,-1.775123,0.595859,-0.168895,1.439142,-0.949513,0.608889,-0.142655,-0.061659,0.631635
14103,-0.778276,0.149153,-1.737638,0.407795,-0.306424,1.453435,-1.245901,0.149894,-0.259855,0.129933,0.590922
14104,-0.836324,0.271862,-2.057289,-0.567824,-1.181257,2.382394,-1.999473,0.981661,0.098194,-0.161432,1.118377
14105,-0.659095,-0.309000,-2.119503,-0.817443,-1.567503,2.852538,-2.087740,-0.441128,-0.681110,-0.644168,0.845002


Correlation Exploration

[[0.37454012 0.95071431 0.73199394 ... 0.60111501 0.70807258 0.02058449]
 [0.96990985 0.83244264 0.21233911 ... 0.29122914 0.61185289 0.13949386]
 [0.29214465 0.36636184 0.45606998 ... 0.60754485 0.17052412 0.06505159]
 ...
 [0.11047411 0.35462216 0.28723899 ... 0.42777313 0.38432665 0.67964728]
 [0.21825389 0.94996118 0.78634501 ... 0.61341139 0.16703395 0.99116863]
 [0.2316717  0.94273177 0.64964665 ... 0.18643826 0.77958447 0.35012526]]


Text(0.5, 1.0, 'Fraud Rand Data 30 epoch')

In [ ]:
np.random.seed(42)
# Create a random 13-column array with 100 rows
random_array = np.random.rand(100, 11)
random_array_df=pd.DataFrame(random_array)
# Print the array
print(random_array)
# Dimension of the covariance matrix
n = 10
fig, ax = plt.subplots(1, 3, figsize=(20, 6))
sns.heatmap(credit_card_data.corr(), annot=True, ax=ax[0], cmap="Blues")
sns.heatmap(new_data.corr(), annot=True, ax=ax[1], cmap="Blues")
sns.heatmap(random_array_df.corr(), annot=True, ax=ax[2], cmap="Blues")
ax[0].set_title("Fraud Original Data 30 epoch")
ax[1].set_title("Fraud GAN Data 30 epoch")
ax[2].set_title("Fraud Rand Data 30 epoch")

Fake Data Target Prepraration 

In [ ]:
target_gan_data=y
target_gan_data_df=pd.DataFrame(target_gan_data)
target_gan_data_df

,0
0,1
1,1
2,1
3,1
4,1
...,...
28208,0
28209,0
28210,0
28211,0


1 in 8 fake are ID'd for training

Switch to change ratio of identified fake data

In [ ]:
#comment out the target_gan_data_df['1'] variable changer you do not wish to happen and uncomment the other. i % 2 is the amount out of each entry you would like to be identified
#target_gan_data_df['1']=[1 if i % 2 == 0 else 0 for i in range(len(target_gan_data_df[0]))]
target_gan_data_df['1']=0
print(target_gan_data_df)
target_gan_data_df['1']


       0  1
0      1  0
1      1  0
2      1  0
3      1  0
4      1  0
...   .. ..
28208  0  0
28209  0  0
28210  0  0
28211  0  0
28212  0  0

[28213 rows x 2 columns]


0        0
1        0
2        0
3        0
4        0
        ..
28208    0
28209    0
28210    0
28211    0
28212    0
Name: 1, Length: 28213, dtype: int64

In [ ]:
target_pre_gan_data=target_gan_data_df[0].values
print(target_pre_gan_data)
target_post_gan_data=target_gan_data_df['1'].values
print(target_post_gan_data)
X = (credit_card_data).values
y_holder=(csv_entry_data["isFraud"]).values
new_data_df=pd.DataFrame(new_data)
#strip data and get only the values
new_entry_data_for_gan=pd.DataFrame(entry_data_for_gan[:].values)
print(new_entry_data_for_gan)
#get the combined set of target and data itself
result_df = pd.concat([new_entry_data_for_gan, new_data], ignore_index=True)
print(result_df)
target_result_df = pd.concat([target_gan_data_df[0], target_gan_data_df['1']], ignore_index=True)
print(target_result_df)

[1 1 1 ... 0 0 0]
[0 0 0 ... 0 0 0]
        0         1         2         3         4         5    6    7    8   \
0      1.0 -0.354846 -0.391393 -0.294012 -0.267946 -0.356414  0.0  0.0  0.0   
1      1.0 -0.354846 -0.391393 -0.294012 -0.260288 -0.356414  0.0  1.0  0.0   
2      1.0 -0.353038 -0.390421 -0.294012 -0.267946 -0.356414  0.0  0.0  0.0   
3      1.0 -0.353038 -0.390421 -0.294012 -0.258473 -0.356414  0.0  1.0  0.0   
4      1.0 -0.341108 -0.384007 -0.294012 -0.267946 -0.356414  0.0  0.0  0.0   
...    ...       ...       ...       ...       ...       ...  ...  ...  ...   
28208  8.0 -0.257298  0.569947  0.950778  0.000553  0.172395  1.0  0.0  0.0   
28209  8.0  0.080382  0.622453  1.238102  0.359858 -0.356414  1.0  0.0  0.0   
28210  8.0 -0.251241  0.856486  1.306561  0.120871 -0.147650  1.0  0.0  0.0   
28211  8.0 -0.318030  0.912248  1.330942 -0.140498 -0.356414  1.0  0.0  0.0   
28212  8.0 -0.328881  0.932106  1.348160  0.133721  0.212849  1.0  0.0  0.0   

        9    10

In [ ]:
new_entry_data_for_gan=pd.DataFrame(entry_data_for_gan[:].values)
print(new_entry_data_for_gan)
result_df = pd.concat([new_entry_data_for_gan, new_data], ignore_index=True)
print(result_df)
target_result_df = pd.concat([target_gan_data_df[0], target_gan_data_df['1']], ignore_index=True)
print(target_result_df)
result_df = pd.concat([new_entry_data_for_gan, new_data], ignore_index=True)
print(result_df)
target_result_df = pd.concat([target_gan_data_df[0], target_gan_data_df['1']], ignore_index=True)
print(target_result_df)

        0         1         2         3         4         5    6    7    8   \
0      1.0 -0.354846 -0.391393 -0.294012 -0.267946 -0.356414  0.0  0.0  0.0   
1      1.0 -0.354846 -0.391393 -0.294012 -0.260288 -0.356414  0.0  1.0  0.0   
2      1.0 -0.353038 -0.390421 -0.294012 -0.267946 -0.356414  0.0  0.0  0.0   
3      1.0 -0.353038 -0.390421 -0.294012 -0.258473 -0.356414  0.0  1.0  0.0   
4      1.0 -0.341108 -0.384007 -0.294012 -0.267946 -0.356414  0.0  0.0  0.0   
...    ...       ...       ...       ...       ...       ...  ...  ...  ...   
28208  8.0 -0.257298  0.569947  0.950778  0.000553  0.172395  1.0  0.0  0.0   
28209  8.0  0.080382  0.622453  1.238102  0.359858 -0.356414  1.0  0.0  0.0   
28210  8.0 -0.251241  0.856486  1.306561  0.120871 -0.147650  1.0  0.0  0.0   
28211  8.0 -0.318030  0.912248  1.330942 -0.140498 -0.356414  1.0  0.0  0.0   
28212  8.0 -0.328881  0.932106  1.348160  0.133721  0.212849  1.0  0.0  0.0   

        9    10  
0      0.0  1.0  
1      0.0  0.0

Evaluation

In [ ]:
print(f"30 epoch run \n")
X_train, X_test, y_train, y_test = train_test_split(new_entry_data_for_gan, y, test_size=0.5, random_state=0,shuffle='true')
gnb_og = GaussianNB()
y_pred = gnb_og.fit(X_train, y_train).predict(X_test)
print(f"Number of mislabeled points out of a total points original dataset:", (X_test.shape[0], (y_test != y_pred).sum()),"error percentage ",((y_test != y_pred).sum())/X_test.shape[0])
X_train, X_test, y_train, y_test = train_test_split(new_data, y[::2], test_size=0.5, random_state=0,shuffle='true')
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(f"Number of mislabeled points out of a total points synethtic data :", (X_test.shape[0], (y_test != y_pred).sum()),"error percentage ",((y_test != y_pred).sum())/X_test.shape[0])
X_train, X_test, y_train, y_test = train_test_split(result_df,target_result_df[0:len(result_df)] , test_size=0.5, random_state=0, shuffle='true')
gnb_gan = GaussianNB()
y_pred = gnb_gan.fit(X_train, y_train).predict(X_test)
rep_value=(len(data_og))
y_test[y_test.index > rep_value]=1
print(f"Number of mislabeled points out of a total points combined dataset:", (X_test.shape[0], (y_test != y_pred).sum()), "error percentage ",((y_test != y_pred).sum())/X_test.shape[0])

30 epoch run 

Number of mislabeled points out of a total points original dataset: (14107, 2617) error percentage  0.18551073934925924
Number of mislabeled points out of a total points synethtic data : (7054, 2015) error percentage  0.28565352991210663
Number of mislabeled points out of a total points combined dataset: (21160, 9712) error percentage  0.4589792060491493


In [ ]:
print("DL model based on the original data")
X_train, X_test, y_train, y_test = train_test_split(entry_data_for_gan, y, test_size=0.5, random_state=0)
dl_og_model = tf.keras.Sequential([
    layers.Dense(32, activation='linear', input_shape=(X_train.shape[1],)),
    layers.Dense(8, activation='gelu'),layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
dl_og_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
dl_og_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
test_loss, test_accuracy = dl_og_model.evaluate(X_test, y_test)
print(f'Epoch Loss Original Data: {test_loss}, Accuracy of Epoch Original Data: {test_accuracy}')


DL model based on the original data
Epoch 1/5


c:\Programming\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7932 - loss: 19.6790 - val_accuracy: 0.9697 - val_loss: 0.2937
Epoch 2/5
221/221 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9763 - loss: 0.2662 - val_accuracy: 0.9836 - val_loss: 0.2254
Epoch 3/5
221/221 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9852 - loss: 0.2108 - val_accuracy: 0.9887 - val_loss: 0.1868
Epoch 4/5
221/221 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9883 - loss: 0.1770 - val_accuracy: 0.9908 - val_loss: 0.1588
Epoch 5/5
221/221 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9902 - loss: 0.1516 - val_accuracy: 0.9924 - val_loss: 0.1363
441/441 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.9926 - loss: 0.1354
Epoch Loss Original Data: 0.13636194169521332, Accuracy of Epoch Original Data: 0.9924151301383972


In [ ]:
print("DL model based on the synthetic data 30 epoch run")
X_train, X_test, y_train, y_test = train_test_split(new_data, y[::2], test_size=0.5, random_state=0)
dl_gan_model = tf.keras.Sequential([
    layers.Dense(32, activation='linear', input_shape=(X_train.shape[1],)),
    layers.Dense(8, activation='gelu'),layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
dl_gan_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
dl_gan_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
test_loss, test_accuracy = dl_gan_model.evaluate(X_test, y_test)
print(f'Epoch Loss Gan Data 30 Epoch: {test_loss}, Accuracy of Epoch Gan Data 30 Epoch: {test_accuracy}')


DL model based on the synthetic data 30 epoch run


Epoch 1/5


c:\Programming\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6248 - loss: 0.6755 - val_accuracy: 0.7143 - val_loss: 0.6055
Epoch 2/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7119 - loss: 0.6079 - val_accuracy: 0.7143 - val_loss: 0.6026
Epoch 3/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7119 - loss: 0.6055 - val_accuracy: 0.7143 - val_loss: 0.6015
Epoch 4/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7119 - loss: 0.6043 - val_accuracy: 0.7143 - val_loss: 0.6009
Epoch 5/5
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7119 - loss: 0.6034 - val_accuracy: 0.7143 - val_loss: 0.6004
221/221 ━━━━━━━━━━━━━━━━━━━━ 0s 452us/step - accuracy: 0.7151 - loss: 0.6003
Epoch Loss Gan Data 30 Epoch: 0.6009892821311951, Accuracy of Epoch Gan Data 30 Epoch: 0.7143464684486389


In [20]:
print("DL model based on the original data 30 epoch run")
keras.utils.set_random_seed(124)
X_train, X_test, y_train, y_test = train_test_split(result_df, target_result_df[0:len(result_df)], test_size=0.5, random_state=0)

rep_value=(len(data_og))
y_test[y_test.index > rep_value]=1
dl_com_gan_model = tf.keras.Sequential([
    layers.Dense(32, activation='linear', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
dl_com_gan_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
dl_com_gan_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
test_loss, test_accuracy = dl_com_gan_model.evaluate(X_test, y_test)
print(f'Epoch Loss Combined Data 30 Epoch: {test_loss}, Accuracy of Epoch Combined Data 30 Epoch: {test_accuracy}')


DL model based on the original data 30 epoch run
Epoch 1/5


c:\Programming\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


331/331 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6992 - loss: 0.6506 - val_accuracy: 0.4714 - val_loss: 3.1047
Epoch 2/5
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8082 - loss: 0.2047 - val_accuracy: 0.6578 - val_loss: 4.4041
Epoch 3/5
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9942 - loss: 0.1149 - val_accuracy: 0.6613 - val_loss: 4.5677
Epoch 4/5
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9958 - loss: 0.0198 - val_accuracy: 0.6618 - val_loss: 4.5367
Epoch 5/5
331/331 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9959 - loss: 0.0185 - val_accuracy: 0.6619 - val_loss: 4.5440
662/662 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.6642 - loss: 4.5519
Epoch Loss Combined Data 30 Epoch: 4.543756008148193, Accuracy of Epoch Combined Data 30 Epoch: 0.6618620157241821
